Notes:
- new data from remm se output
- old data from wfrc data portal
- before running I standarized the column names
- Filter city areas

Create a Feature Layer for each combination of Map Display Geography (4) x Variables (7) wherein the attribute fields are 

  Like this for Households (for the other 6 variables substitute hh_ with: pop_, ijb_, ojb, rjb_, tjb_,  hji_)

TAZID, city, county, med_dist, small_dist, dev_acres, hh_19, hh_20, hh_21…hh_50…old_hh_20, old_hh_25, old_hh_50
Sml_DistID, county, dev_acres, hh_19, hh_20, hh_21…hh_50…old_hh_20, old_hh_25, old_hh_50
Med_DistID, county, dev_acres, hh_19, hh_20, hh_21…hh_50…old_hh_20, old_hh_25, old_hh_50
CityID, county, dev_acres, hh_19, hh_20, hh_21…hh_50…old_hh_20, old_hh_25, old_hh_50


### For the map:
<!-- 1. obtain taz 832 and 900 shapefile -->
<!-- 2. attributes we need:
  - total households
  - total population
  - total jobs
  - industrial jobs(5,10) 
  - retail jobs(1,9) 
  - office jobs(6,4,3)
  - typical jobs (sum of three types, calculated)
  - HJ Intensity (this is (HH * 1.8) + Jobs) -->
<!-- 3. get taz outout from 2019 REMM -->
<!-- 4. get Official TAZ output from 2015 REMM (confirm TAZ version) -->
<!-- 5. Generate folder structure -->
6. Apportion new taz data to:
  - taz 9.0.0
  - medium district
  - city
  - small district (later)
7. Apportion old taz data to:
  - taz 9.0.0
  - medium district
  - city
  - small district (later)
8. create shapefiles/feature classes for each attribute (see #2)

### For the chart:
1. create a json for each record using the following geographies:
  - taz
  - medium district
  - city
  - small district (later)



In [1]:
import arcpy
from arcpy import env
import os
import glob
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [2]:
# import datasets
# parcels = r'.\\Inputs\\remm_base_year_20220513.gdb\\parcels_2019'
taz832 = r".\Inputs\TAZ_832_ID_Only.shp"
taz900 = r".\Inputs\TAZ_900.shp"
cities = r".\Inputs\CityArea_UTM12.shp"
# small_districts = ???
# regions_df = pd.DataFrame.spatial.from_featureclass(regions_shp)

taz832_sdf = pd.DataFrame.spatial.from_featureclass(taz832)
taz900_sdf = pd.DataFrame.spatial.from_featureclass(taz900)

In [3]:
pm_folder = r".\New_SE_Data"
new_taz_se = glob.glob(os.path.join(pm_folder,'SE_WF_*.csv'))
len(new_taz_se)

32

In [4]:
old_se_folder = r".\Old_SE_Data"
old_taz_se = glob.glob(os.path.join(old_se_folder,'*_TAZ.csv'))
old_city_se = glob.glob(os.path.join(old_se_folder,'*_City_Area.csv'))

In [5]:
outputs = r'.\Outputs'
if not os.path.exists(outputs):
    os.makedirs(outputs)

scratch = os.path.join(outputs, 'scratch.gdb')
if not arcpy.Exists(scratch):
    arcpy.CreateFileGDB_management(outputs, 'scratch.gdb')

map_folder = os.path.join(outputs, "map")
if not os.path.exists(map_folder):
    os.makedirs(map_folder)

chart_folder = os.path.join(outputs, "chart")
if not os.path.exists(chart_folder):
    os.makedirs(chart_folder)

In [6]:
# year = os.path.splitext(os.path.basename(new_taz_se[0]))[0].split('_')[-1]
# x = pd.read_csv(new_taz_se[0])
# x['OFFI'] = x['OFFI'] + x['GVED'] + x['HLTH']
# x['TPCL'] = x['OFFI'] + x['RETEMP'] + x['INDEMP']
# x['HJI'] = x['TOTHH']*1.8 + x['TOTEMP']
# x = x[[';TAZID', 'TOTHH', 'HHPOP', 'TOTEMP','RETEMP', 'INDEMP', 'OFFI', 'TPCL', 'HJI']].copy()
# x.columns = ['TAZID', f'HH_{year}', f'POP_{year}', f'EMP_{year}', f'RTL_{year}', f'IND_{year}', f'OFFI_{year}', f'TPCL_{year}', f'HJI_{year}']
# x


### Process New SE Data

In [7]:
base = taz832_sdf[['TAZID','SHAPE']].copy()
base.shape

for csv in new_taz_se:

    year = os.path.splitext(os.path.basename(csv))[0].split('_')[-1]
    df = pd.read_csv(csv)
    df['OFFI'] = df['OFFI'] + df['GVED'] + df['HLTH']
    df['TPCL'] = df['OFFI'] + df['RETEMP'] + df['INDEMP']
    df['HJI'] = df['TOTHH']*1.8 + df['TOTEMP']
    df = df[[';TAZID', 'TOTHH', 'HHPOP', 'TOTEMP','RETEMP', 'INDEMP', 'OFFI', 'TPCL', 'HJI']].copy()
    df.columns = ['TAZID', f'HH_{year}', f'POP_{year}', f'EMP_{year}', f'RTL_{year}', f'IND_{year}', f'OFFI_{year}', f'TPCL_{year}', f'HJI_{year}']
    
    base = base.merge(df, on='TAZID', how='left')

new_se = base
new_se.rename({'TAZID': 'TAZID832'}, axis=1, inplace=True)

In [8]:
new_se.shape

(2881, 258)

In [9]:
# export
new_se_taz_832 = os.path.join(scratch, '_01_new_se_taz_832')
new_se.spatial.to_featureclass(location=new_se_taz_832,sanitize_columns=False)

'e:\\Projects\\REMM-Process-Progression-Metrics-For-Web\\Outputs\\scratch.gdb\\_01_new_se_taz_832'

### Process Old SE Data

In [10]:
se_dict =  {'TOTHH':'HH',    
            'HHPOP':'POP',
            'RETEMP':'RTL',
            'TOTEMP':'TPCL',
            'ALLEMP':'EMP',
            'INDEMP':'IND',
            'OTHEMP':'OFFI'}

In [11]:
# test = pd.read_csv(old_taz_se[0])
# category = se_dict[test.iloc[0]['SECategory']]
# val_cols = [col for col in list(test.columns) if 'YEAR' in col and  'D' not in col and int(col[4:]) >= 2019]
# test = test[['CO_TAZID'] + val_cols]
# new_val_cols = [col.replace('YEAR',category + '_') for col in val_cols]
# test.columns = ['CO_TAZID'] + new_val_cols
# test

In [12]:
old_base = taz832_sdf[['CO_TAZID','SHAPE']].copy()
old_base.shape

for csv in old_taz_se:

    df = pd.read_csv(csv)
    category = se_dict[df.iloc[0]['SECategory']]
    val_cols = [col for col in list(df.columns) if 'YEAR' in col and  'D' not in col and int(col[4:]) >= 2019]
    df = df[['CO_TAZID'] + val_cols]
    new_val_cols = [col.replace('YEAR',category + '_') for col in val_cols]
    df.columns = ['CO_TAZID'] + new_val_cols
    old_base = old_base.merge(df, on='CO_TAZID',how='left')

old_se = old_base
old_se.rename({'CO_TAZID': 'COTAZID832'}, axis=1, inplace=True)

In [13]:
# calculate household jobs index
for year in range(2019,2051):
    old_se['HJI_{}'.format(year)] = (old_se['HH_{}'.format(year)] * 1.8) + old_se['EMP_{}'.format(year)]

In [14]:
old_se.shape

(2881, 258)

In [15]:
# export
old_se_taz_832 = os.path.join(scratch, '_02_old_se_taz_832')
old_se.spatial.to_featureclass(location=old_se_taz_832,sanitize_columns=False)

'e:\\Projects\\REMM-Process-Progression-Metrics-For-Web\\Outputs\\scratch.gdb\\_02_old_se_taz_832'

## Identity

In [16]:
# compare output dimensions
old_se.shape == new_se.shape

True

In [17]:
# merge geometry between taz 832 and output geometries (taz900, )
taz900_identity = arcpy.Identity_analysis(taz900, new_se_taz_832, os.path.join(scratch, '_03_taz832_taz900_identity'))
arcpy.management.DeleteField(taz900_identity,['TAZID', 'DISTMED'], "KEEP_FIELDS")

cities_identity = arcpy.Identity_analysis(cities, new_se_taz_832,os.path.join(scratch, '_03_taz832_cities_identity'))
arcpy.management.DeleteField(cities_identity,'CityArea', "KEEP_FIELDS")

<Result '.\\Outputs\\scratch.gdb\\_03_taz832_cities_identity'>

## Apportion the attributes from old geometry to the new ones

- The Apportion tool in arcpy is still lame 
- we have to run a command (apportion_command_for_arcgis_pro.txt) in the arcgis pro python window to run it

In [76]:
os.startfile(r".\REMM-Process-Progression-Metrics-For-Web.aprx")

In [18]:
# apportion = arcpy.ApportionPolygon_analysis(new_se_taz_832, app_columns, identity, os.path.join(outputs, 'new_se_apportion_to_taz900.shp'), "AREA", "", "", "MAINTAIN_GEOMETRIES")
new_taz_apportion = os.path.join(scratch,'_05_new_se_apportion_to_taz900')
new_city_apportion = os.path.join(scratch,'_05_new_se_apportion_to_cities')
old_taz_apportion = os.path.join(scratch,'_05_old_se_apportion_to_taz900')
old_city_apportion = os.path.join(scratch,'_05_old_se_apportion_to_cities')

## Dissolve to get to desired geometry

In [19]:
app_columns = list(new_se.columns)[2:]
stat_fields = [[col,'SUM'] for col in app_columns]

#########
# New
#########

# taz (9.0.0)
new_se_taz900_dissolve = arcpy.Dissolve_management(new_taz_apportion, os.path.join(scratch, '_06_new_se_taz900_dissolve'),
                          'TAZID', stat_fields, "MULTI_PART")

# distmed (9.0.0)
new_se_distmed_dissolve = arcpy.Dissolve_management(new_taz_apportion, os.path.join(scratch, '_06_new_se_distmed_dissolve'),
                          'DISTMED', stat_fields, "MULTI_PART")

# city area
new_se_cityarea_dissolve = arcpy.Dissolve_management(new_city_apportion, os.path.join(scratch, '_06_new_se_cityarea_dissolve'),
                          'CITYAREA', stat_fields, "MULTI_PART")




In [20]:
#########
# Old
#########

# taz (9.0.0)
old_se_taz900_dissolve = arcpy.Dissolve_management(old_taz_apportion, os.path.join(scratch, '_06_old_se_taz900_dissolve'),
                          'TAZID', stat_fields, "MULTI_PART")

# distmed (9.0.0)
old_se_distmed_dissolve = arcpy.Dissolve_management(old_taz_apportion, os.path.join(scratch, '_06_old_se_distmed_dissolve'),
                          'DISTMED', stat_fields, "MULTI_PART")

# city area
old_se_cityarea_dissolve = arcpy.Dissolve_management(old_city_apportion, os.path.join(scratch, '_06_old_se_cityarea_dissolve'),
                          'CityArea', stat_fields, "MULTI_PART")


In [48]:
# read in new dataframes
new_se_taz900_dissolve_df = pd.DataFrame.spatial.from_featureclass(new_se_taz900_dissolve[0])
new_se_distmed_dissolve_df = pd.DataFrame.spatial.from_featureclass(new_se_distmed_dissolve[0])
new_se_cityarea_dissolve_df = pd.DataFrame.spatial.from_featureclass(new_se_cityarea_dissolve[0])
old_se_taz900_dissolve_df = pd.DataFrame.spatial.from_featureclass(old_se_taz900_dissolve[0])
old_se_distmed_dissolve_df = pd.DataFrame.spatial.from_featureclass(old_se_distmed_dissolve[0])
old_se_cityarea_dissolve_df = pd.DataFrame.spatial.from_featureclass(old_se_cityarea_dissolve[0])


In [50]:
# Remove 'SUM' from column names
to_replace = ['SUM_'+ col for col in app_columns]
replace_dict = dict(zip(to_replace, app_columns))

new_se_taz900_dissolve_df.rename(replace_dict, axis=1, inplace=True)
old_se_taz900_dissolve_df.rename(replace_dict, axis=1, inplace=True)

new_se_distmed_dissolve_df.rename(replace_dict, axis=1, inplace=True)
old_se_distmed_dissolve_df.rename(replace_dict, axis=1, inplace=True)

new_se_cityarea_dissolve_df.rename(replace_dict, axis=1, inplace=True)
old_se_cityarea_dissolve_df.rename(replace_dict, axis=1, inplace=True)

In [127]:
#====================
# Process TAZ
#====================

taz_folder = os.path.join(map_folder, "TAZ")
if not os.path.exists(taz_folder):
    os.makedirs(taz_folder)

categories = ['HH', 'POP', 'EMP', 'RTL', 'IND', 'OFFI', 'TPCL', 'HJI']
for c in categories:
    new_cols = [col for col in app_columns if col.split('_')[0] == c]
    new_temp_df = new_se_taz900_dissolve_df[['TAZID','SHAPE'] + new_cols].copy()
    old_temp_df = old_se_taz900_dissolve_df[['TAZID','SHAPE'] + new_cols].copy()
    

    new_new_names = [col.replace(c, 'N') for col in new_cols]
    new_old_names = [col.replace(c, 'O') for col in new_cols]
    new_rename_dict = dict(zip(new_cols, new_new_names))
    old_rename_dict = dict(zip(new_cols, new_old_names))
    new_temp_df.rename(new_rename_dict, axis=1, inplace=True)
    old_temp_df.rename(old_rename_dict, axis=1, inplace=True)

    merged = new_temp_df.merge(old_temp_df, on=['TAZID','SHAPE'], how='left')

    merged[new_new_names + new_old_names] = merged[new_new_names + new_old_names].fillna(value=0)

    outfile = os.path.join(taz_folder, '{}_PROJECTIONS_by_TAZ.shp'.format(c))
    merged.spatial.to_featureclass(location=outfile, sanitize_columns=False)

    arcpy.AddField_management(outfile, "ACRES", "FLOAT")
    exp = "round(!SHAPE.AREA@ACRES!,2)"
    arcpy.CalculateField_management(outfile, "ACRES", exp, "PYTHON3")

    # convert to json
    category_folder = os.path.join(chart_folder, 'TAZ', c)
    if not os.path.exists(category_folder):
        os.makedirs(category_folder)

    for index, row in merged.iterrows():
        f = open(os.path.join(category_folder, "TAZ_{}.json".format(row['TAZID'])), "a")
        cols = [col for col in  merged.columns if col.split('_')[0]=='N']
    
        f.write("[\n")

        for col in cols:
            f.write("\t{\n")
            f.write('''\t\t"C":"{}",\n'''.format(c))
            f.write('''\t\t"Y":{},\n'''.format(col.split('_')[1]))
            f.write('''\t\t"V":{}\n'''.format(row[col]))
            
            if col.split('_')[1] != "2050":
                f.write("\t},\n")
            else:
                f.write("\t}\n")

        f.write("]\n")
        f.close()

In [128]:
#====================
# Process DISTMED
#====================

distmed_folder = os.path.join(map_folder, "DISTMED")
if not os.path.exists(distmed_folder):
    os.makedirs(distmed_folder)

categories = ['HH', 'POP', 'EMP', 'RTL', 'IND', 'OFFI', 'TPCL', 'HJI']
for c in categories:
    new_cols = [col for col in app_columns if col.split('_')[0] == c]
    new_temp_df = new_se_distmed_dissolve_df[['DISTMED','SHAPE'] + new_cols].copy()
    old_temp_df = old_se_distmed_dissolve_df[['DISTMED','SHAPE'] + new_cols].copy()
    

    new_new_names = [col.replace(c, 'N') for col in new_cols]
    new_old_names = [col.replace(c, 'O') for col in new_cols]
    new_rename_dict = dict(zip(new_cols, new_new_names))
    old_rename_dict = dict(zip(new_cols, new_old_names))
    new_temp_df.rename(new_rename_dict, axis=1, inplace=True)
    old_temp_df.rename(old_rename_dict, axis=1, inplace=True)

    merged = new_temp_df.merge(old_temp_df, on=['DISTMED','SHAPE'], how='left')

    merged[new_new_names + new_old_names] = merged[new_new_names + new_old_names].fillna(value=0)
    merged[new_new_names + new_old_names] = merged[new_new_names + new_old_names].round(0)

    outfile = os.path.join(distmed_folder, '{}_PROJECTIONS_by_DISTMED.shp'.format(c))
    merged.spatial.to_featureclass(location=outfile, sanitize_columns=False)

    arcpy.AddField_management(outfile, "ACRES", "FLOAT")
    exp = "round(!SHAPE.AREA@ACRES!,2)"
    arcpy.CalculateField_management(outfile, "ACRES", exp, "PYTHON3")

    # convert to json
    category_folder = os.path.join(chart_folder, 'DISTMED', c)
    if not os.path.exists(category_folder):
        os.makedirs(category_folder)

    for index, row in merged.iterrows():
        f = open(os.path.join(category_folder, "DISTMED_{}.json".format(row['DISTMED'])), "a")
        cols = [col for col in  merged.columns if col.split('_')[0]=='N']
    
        f.write("[\n")

        for col in cols:
            f.write("\t{\n")
            f.write('''\t\t"C":"{}",\n'''.format(c))
            f.write('''\t\t"Y":{},\n'''.format(col.split('_')[1]))
            f.write('''\t\t"V":{}\n'''.format(row[col]))
            
            if col.split('_')[1] != "2050":
                f.write("\t},\n")
            else:
                f.write("\t}\n")

        f.write("]\n")
        f.close()


In [129]:
#====================
# Process City Area
#====================

cityarea_folder = os.path.join(map_folder, "CITYAREA")
if not os.path.exists(cityarea_folder):
    os.makedirs(cityarea_folder)

categories = ['HH', 'POP', 'EMP', 'RTL', 'IND', 'OFFI', 'TPCL', 'HJI']
for c in categories:
    new_cols = [col for col in app_columns if col.split('_')[0] == c]
    new_temp_df = new_se_cityarea_dissolve_df[['CityArea','SHAPE'] + new_cols].copy()
    old_temp_df = old_se_cityarea_dissolve_df[['CityArea','SHAPE'] + new_cols].copy()
    

    new_new_names = [col.replace(c, 'N') for col in new_cols]
    new_old_names = [col.replace(c, 'O') for col in new_cols]
    new_rename_dict = dict(zip(new_cols, new_new_names))
    old_rename_dict = dict(zip(new_cols, new_old_names))
    new_temp_df.rename(new_rename_dict, axis=1, inplace=True)
    old_temp_df.rename(old_rename_dict, axis=1, inplace=True)

    merged = new_temp_df.merge(old_temp_df, on=['CityArea','SHAPE'], how='left')

    merged[new_new_names + new_old_names] = merged[new_new_names + new_old_names].fillna(value=0)


    outfile = os.path.join(distmed_folder, '{}_PROJECTIONS_by_CITYAREA.shp'.format(c))
    merged.spatial.to_featureclass(location=outfile, sanitize_columns=False)

    arcpy.AddField_management(outfile, "ACRES", "FLOAT")
    exp = "round(!SHAPE.AREA@ACRES!,2)"
    arcpy.CalculateField_management(outfile, "ACRES", exp, "PYTHON3")

    # convert to json
    category_folder = os.path.join(chart_folder, 'CITYAREA', c)
    if not os.path.exists(category_folder):
        os.makedirs(category_folder)

    for index, row in merged.iterrows():
        f = open(os.path.join(category_folder, "CITYAREA_{}.json".format(row['CityArea'])), "a")
        cols = [col for col in  merged.columns if col.split('_')[0]=='N']
    
        f.write("[\n")

        for col in cols:
            f.write("\t{\n")
            f.write('''\t\t"C":"{}",\n'''.format(c))
            f.write('''\t\t"Y":{},\n'''.format(col.split('_')[1]))
            f.write('''\t\t"V":{}\n'''.format(row[col]))
            
            if col.split('_')[1] != "2050":
                f.write("\t},\n")
            else:
                f.write("\t}\n")

        f.write("]\n")
        f.close()


In [130]:
# category_folder = os.path.join(chart_folder, c)
# if not os.path.exists(category_folder):
#     os.makedirs(category_folder)

# m = merged.copy()
# del m['SHAPE']
# m.to_json('file.json', orient='records', lines=True)
  

In [103]:
# import arcgis.features
# from arcgis.features import GeoSeriesAccessor
# gsa = arcgis.features.GeoSeriesAccessor(new_temp_df['SHAPE'])
# new_temp_df['the_area'] = gsa.area
# new_temp_df

,CityArea,SHAPE,N_2019,N_2020,N_2021,N_2022,N_2023,N_2024,N_2025,N_2026,N_2027,N_2028,N_2029,N_2030,N_2031,N_2032,N_2033,N_2034,N_2035,N_2036,N_2037,N_2038,N_2039,N_2040,N_2041,N_2042,N_2043,N_2044,N_2045,N_2046,N_2047,N_2048,N_2049,N_2050,the_area
0,Alpine,"{""rings"": [[[-12442943.7757, 4937264.529899999...",6435.200004,6538.200003,6946.400004,7218.600005,7477.200006,7625.000006,7723.200006,7832.000006,7910.200006,7958.400006,8034.000006,8130.800006,8212.800006,8277.800006,8339.000006,8409.400006,8492.400006,8568.200006,8661.000006,8767.400006,8854.600006,8945.600006,9052.600006,9190.400006,9299.400006,9423.200005,9531.600005,9615.400005,9697.600005,9784.000005,9850.600005,9891.200005,30776463.794043
1,Alta,"{""rings"": [[[-12426519.0222, 4953174.323600002...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,144892726.603075
2,Altamont,"{""rings"": [[[-12276835.5724, 4919352.339000002...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13468399.267567
3,Alton,"{""rings"": [[[-12520939.344700001, 4500318.2993...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9828588.953654
4,Amalga,"{""rings"": [[[-12457630.286, 5144333.3347999975...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26460753.572179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,Willard,"{""rings"": [[[-12472854.2697, 5078290.666500002...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21839566.625954
294,Woodland Hills,"{""rings"": [[[-12429225.6549, 4869928.106600001...",739.799999,772.199999,791.999999,813.599999,829.799999,844.199999,853.199999,865.799999,878.399999,885.599999,901.799999,914.399999,921.599999,923.399999,932.399999,943.199999,957.599999,971.999999,998.999999,1022.399998,1047.599998,1074.599998,1108.799998,1168.199998,1231.199998,1324.799998,1427.399998,1515.599997,1609.199997,1686.599997,1726.199997,1776.599997,11314045.360356
295,Woodruff,"{""rings"": [[[-12374594.4946, 5090375.112000003...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2125805.706251
296,Woods Cross,"{""rings"": [[[-12459834.5079, 4995266.601899996...",14516.199956,14212.999959,14843.799954,15077.799953,15157.199953,15520.399948,16105.599941,16424.399938,16705.799935,16977.599932,17291.399930,17547.999929,17700.999928,17811.599927,17913.799927,18078.199926,18183.399925,18249.199925,18353.599924,18488.399924,18697.399923,18865.599922,18998.999922,19118.999921,19391.199921,19582.599921,19735.199921,19882.999920,20172.599920,20234.199921,20686.799921,21157.199919,18266783.84168


In [119]:
# a = m.iloc[0]
# cols = [col for col in  m.columns if col.split('_')[0]=='N']
# f = open("CITYAREA_{}.json".format(a['CityArea']), "a")


# f.write("[\n")


# for col in cols:
#     f.write("\t{\n")
#     f.write('''\t\t"C":"{}",\n'''.format(c))
#     f.write('''\t\t"Y":{},\n'''.format(col.split('_')[1]))
#     f.write('''\t\t"V":{}\n'''.format(a[col]))
    
#     if col.split('_')[1] != "2050":
#         f.write("\t},\n")
#     else:
#         f.write("\t}\n")

# f.write("]\n")



# f.close()

In [113]:
c

'HJI'